In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
.master("local[*]") \
.appName("test") \
.getOrCreate()

24/01/21 13:51:33 WARN Utils: Your hostname, codespaces-fce2a9 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/01/21 13:51:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/21 13:51:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/raw/green')

In [4]:
df_green.createOrReplaceTempView('green')

In [5]:
df_green_revenue =  spark.sql(
    """
    SELECT  
         date_trunc('hour', lpep_pickup_datetime) AS hour,
         PULocationID AS zone,
         

         SUM(total_amount) AS amount,
         COUNT(1) AS number_records
    FROM
        green
    WHERE
         lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 
        1,2
    ORDER BY
        1,2
    """
) 

In [7]:
df_green_revenue.write.parquet('data/raw/report/revenue/green', mode='overwrite')

In [8]:
spark

In [9]:
df_yellow = spark.read.parquet('data/raw/yellow')

In [10]:
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       2| 2020-01-06 09:18:38|  2020-01-06 09:33:56|              1|         3.03|         1|                 N|         263|         233|           1|       12.0|  0.0|    0.5|       1.5|         0.0|                  0.3

In [11]:
df_yellow.createOrReplaceTempView('yellow')

In [12]:
df_yellow_revenue = spark.sql(

 """
    SELECT  
         date_trunc('hour', tpep_pickup_datetime) AS hour,
         PULocationID AS zone,
         

         SUM(total_amount) AS amount,
         COUNT(1) AS number_records
    FROM
        yellow
    WHERE
         tpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 
        1,2
    ORDER BY
        1,2
    """

)

In [13]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   3|              25.0|             1|
|2020-01-01 00:00:00|   4|1004.3000000000002|            57|
|2020-01-01 00:00:00|   7|455.16999999999996|            38|
|2020-01-01 00:00:00|  10|             42.41|             2|
|2020-01-01 00:00:00|  12|             107.0|             6|
|2020-01-01 00:00:00|  13|1214.8000000000002|            56|
|2020-01-01 00:00:00|  14|               8.8|             1|
|2020-01-01 00:00:00|  15|             34.09|             1|
|2020-01-01 00:00:00|  17|            220.21|             8|
|2020-01-01 00:00:00|  18|               5.8|             1|
|2020-01-01 00:00:00|  24|            754.95|            45|
|2020-01-01 00:00:00|  25|            324.35|            16|
|2020-01-01 00:00:00|  32|              18.0|             1|
|2020-01-01 00:00:00|  3

In [ ]:
df_yellow_revenue.repartition(20).write.parquet('data/raw/')